In [3]:
#Obtaining Data
import wget
from pycocotools.coco import COCO
import urllib.request

#Computer Vision
import cv2
import numpy as np

#Storage
import pickle
import os

In [4]:
#Global Variables
jsonfolder = "./cocodataset/"
imagefolder = "./cocoimages/"
datafolder = "./tempdata/"

imageHeight = 500
imageWidth = 500
maxCount = 100

In [5]:
#Load Image Ids
coco=COCO(jsonfolder+'instances_train2017.json')
cats = ['person','bicycle','car','motorcycle','bus', 'truck']
catIds = coco.getCatIds(catNms=cats)
catImgs = {c:coco.getImgIds( catIds=coco.getCatIds(catNms=[c]) ) for c in cats}

loading annotations into memory...
Done (t=18.16s)
creating index...
index created!


In [2]:
#Verification
catIds = coco.getCatIds(catNms=cats)
print(catIds)
imgs = coco.loadImgs(catImgs["person"])
print(imgs[0]['id'])
annIds = coco.getAnnIds(imgIds=imgs[0]['id'], iscrowd=None)
anns = coco.loadAnns(annIds)
# print(anns[0])
print(anns[0]['bbox'])
val = [anns[i]['category_id'] for i in range(len(anns))]
print(val)

NameError: name 'coco' is not defined

In [4]:
#Store all images as files

if not os.path.exists(datafolder):
    os.makedirs(datafolder)
for k in catImgs:
    print(k)
    imgs = coco.loadImgs(catIds[k])
    dataset = np.ndarray(shape=(len(imgs), imageHeight, imageWidth), dtype=np.float32)
#     test = np.ndarray(shape=(len(imgs), imageHeight, imageWidth), dtype=np.float32)
#     output_data = [np.ndarray(shape=(len(imgs)]
    count = 0
    for i in range(len(imgs)):
        if count > maxCount:
            break
        try:
            resp = urllib.request.urlopen(imgs[i]['coco_url'])
            image = np.asarray(bytearray(resp.read()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            edges = cv2.Canny(image,100,100)
            height, width = edges.shape;
            method = cv2.INTER_CUBIC;
            if width < imageWidth or height < imageHeight:
                method = cv2.INTER_AREA;
            resized = cv2.resize(edges, (imageWidth, imageHeight), interpolation = method)
            dataset[i] = resized;
            
            annIds = coco.getAnnIds(imgIds=imgs[i]['id'], catIds=catIds, iscrowd=None)
            anns = coco.loadAnns(annIds)
        except Exception as e:
            print(e)
        count++
    with open(datafolder+k, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)

person


KeyboardInterrupt: 

In [ ]:
#Read in pickle files
for file in os.listdir(datafolder):
    with open(file, 'rb') as f:
        image_dataset = pickle.load(file)
        category = file.split(".")[0]